In [9]:
import numpy as np 
import pandas as pd
import shap 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import cross_val_score, GridSearchCV 
from sklearn import metrics 
from pandas import read_csv 
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.svm import SVR 
from sklearn.neighbors import KNeighborsRegressor 
from scipy import stats 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score 
from matplotlib.pyplot import savefig
#Read data
krfp = pd.read_csv('krfp.csv')
krfp= krfp.loc[:, (krfp.sum(axis=0) != 0)]
data = pd.read_csv('dataset.csv')
all = pd.concat([krfp,data],axis=1)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
krfp2 = all.copy(deep=True)
krfp2['Inc'] = krfp2['PCE']-krfp2['control_PCE']
krfp2['inc_p'] = krfp2['PCE']/krfp2['control_PCE']
new2 = pd.DataFrame(scaler.fit_transform(krfp2),columns=krfp2.columns)
new2.head()

,KRFP1,KRFP2,KRFP10,KRFP14,KRFP16,KRFP17,KRFP20,KRFP21,KRFP25,KRFP72,...,anchor_end_SH,anchor_end_SiO,MetalOxide_ITO,MetalOxide_NiOx,MetalOxide_SnO2,MetalOxide_TiO2,MetalOxide_ZnO,PCE,Inc,inc_p
0,-0.3386,-0.107521,-0.107521,-0.132068,-0.07581,-0.07581,-0.271329,-0.07581,-0.258985,-0.107521,...,-0.107521,-0.232845,-0.132068,1.752075,-0.994302,-0.508913,-0.188422,0.804697,0.586141,0.212197
1,-0.3386,-0.107521,-0.107521,-0.132068,-0.07581,-0.07581,-0.271329,-0.07581,-0.258985,-0.107521,...,-0.107521,-0.232845,-0.132068,1.752075,-0.994302,-0.508913,-0.188422,0.421904,-0.401313,-0.447286
2,-0.3386,-0.107521,-0.107521,-0.132068,-0.07581,-0.07581,-0.271329,-0.07581,-0.258985,-0.107521,...,-0.107521,-0.232845,-0.132068,1.752075,-0.994302,-0.508913,-0.188422,1.679651,2.702114,1.599597
3,-0.3386,-0.107521,-0.107521,-0.132068,-0.07581,-0.07581,-0.271329,-0.07581,-0.258985,-0.107521,...,-0.107521,-0.232845,-0.132068,-0.570752,-0.994302,1.964971,-0.188422,-0.797563,0.155893,0.243901
4,-0.3386,-0.107521,-0.107521,-0.132068,-0.07581,-0.07581,-0.271329,-0.07581,-0.258985,-0.107521,...,-0.107521,-0.232845,-0.132068,-0.570752,-0.994302,1.964971,-0.188422,-0.704599,0.395703,0.461457


In [11]:
def print_list_in_chunks(list, chunk_size):
  for i in range(0, len(list), chunk_size):
    print(", ".join(list[i:i + chunk_size]))

In [12]:
correlation2 = new2.corr().iloc[:,-1].sort_values(ascending=False) # increase percentage
fp2 = correlation2.loc[(correlation2 > 0.15) & correlation2.index.str.contains("KRFP")]
print(fp2.index)
print_list_in_chunks([f"{i}({v:.5f})" for i , v in zip(fp2.index,fp2.values)], 5)

Index(['KRFP3998', 'KRFP1998', 'KRFP3750', 'KRFP4121', 'KRFP354', 'KRFP350',
       'KRFP1907', 'KRFP3882', 'KRFP4752', 'KRFP2000', 'KRFP3999', 'KRFP4006',
       'KRFP3943', 'KRFP776', 'KRFP362', 'KRFP3740', 'KRFP3733', 'KRFP1926',
       'KRFP398', 'KRFP397', 'KRFP669', 'KRFP476'],
      dtype='object')
KRFP3998(0.25375), KRFP1998(0.24698), KRFP3750(0.24685), KRFP4121(0.22559), KRFP354(0.22559)
KRFP350(0.22559), KRFP1907(0.22354), KRFP3882(0.21931), KRFP4752(0.21100), KRFP2000(0.20708)
KRFP3999(0.20708), KRFP4006(0.20708), KRFP3943(0.20308), KRFP776(0.20128), KRFP362(0.20117)
KRFP3740(0.19478), KRFP3733(0.17441), KRFP1926(0.17104), KRFP398(0.16196), KRFP397(0.15961)
KRFP669(0.15182), KRFP476(0.15168)


In [13]:
correlation3 = new2.corr().iloc[:,-2].sort_values(ascending=False) # increase
fp3 = correlation3.loc[(correlation3 > 0.15) & correlation3.index.str.contains("KRFP")]
print(fp3.index)
print_list_in_chunks([f"{i}({v:.5f})" for i , v in zip(fp3.index,fp3.values)], 5)

Index(['KRFP3750', 'KRFP3882', 'KRFP3943', 'KRFP1849', 'KRFP3991', 'KRFP4740',
       'KRFP3733', 'KRFP1907', 'KRFP354', 'KRFP4121', 'KRFP350', 'KRFP362',
       'KRFP2262', 'KRFP3740', 'KRFP1646', 'KRFP1645'],
      dtype='object')
KRFP3750(0.23931), KRFP3882(0.23597), KRFP3943(0.21278), KRFP1849(0.20485), KRFP3991(0.20485)
KRFP4740(0.20485), KRFP3733(0.20289), KRFP1907(0.19806), KRFP354(0.18398), KRFP4121(0.18398)
KRFP350(0.18398), KRFP362(0.16408), KRFP2262(0.16378), KRFP3740(0.16316), KRFP1646(0.16004)
KRFP1645(0.15591)


In [14]:
correlationPCE = new2.corr().iloc[:,-3].sort_values(ascending=False) # high pce
fp_pce = correlationPCE.loc[(correlationPCE > 0.15) & correlationPCE.index.str.contains("KRFP")]
print(fp_pce.index)
print_list_in_chunks([f"{i}({v:.5f})" for i , v in zip(fp_pce.index,fp_pce.values)], 5)

Index(['KRFP298', 'KRFP4491', 'KRFP1', 'KRFP4591', 'KRFP3789', 'KRFP3957',
       'KRFP3971', 'KRFP2673', 'KRFP1645'],
      dtype='object')
KRFP298(0.18701), KRFP4491(0.17609), KRFP1(0.17243), KRFP4591(0.16957), KRFP3789(0.16520)
KRFP3957(0.16520), KRFP3971(0.16520), KRFP2673(0.16520), KRFP1645(0.15275)
